<a href="https://colab.research.google.com/github/RoyZhaoGA/DroughtForecastAI/blob/main/GNNDroughtPrediction200k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
processed_dataset = pd.read_csv("/content/drive/MyDrive/DroughtCastAI/Dataset/Processed_Datasets/CMIP6_processed_dataset_200km.csv")

In [ ]:
processed_dataset

,Unnamed: 0.1,Unnamed: 0,Month,Lat,Lon,temperature_2m,temperature_2m_min,temperature_2m_max,dewpoint_temperature_2m,total_precipitation_sum,...,NDVI,EVI,next_month_tempreature,next_month_precipitation,spei01,spei03,datetime,year,month2,next_month_soil_moisture_am
0,287,287,201505,-17.068,15.271,297.249,291.684,304.108,292.141,0.000248,...,1557.17,1132.95,303.650,3.938450e-06,-1.030740,-0.937773,2015-05-01,2015,5,0.148753
1,1125,287,201506,-17.068,15.271,298.479,292.966,307.924,293.551,0.000030,...,1507.68,1116.97,302.250,3.481040e-06,-1.983540,-1.747770,2015-06-01,2015,6,0.169415
2,1963,287,201507,-17.068,15.271,300.038,296.358,304.813,295.583,0.039828,...,1814.58,1368.95,302.435,1.429650e-05,-0.773948,-1.519480,2015-07-01,2015,7,0.229645
3,2801,287,201508,-17.068,15.271,300.277,297.167,304.959,297.178,0.201599,...,3233.08,2475.59,303.790,6.708030e-05,1.125420,0.176900,2015-08-01,2015,8,0.226260
4,3639,287,201509,-17.068,15.271,300.453,296.962,304.370,297.498,0.118411,...,3829.11,2849.36,304.714,1.539020e-06,0.593335,0.588032,2015-09-01,2015,9,0.155879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83575,79997,387,202304,51.204,11.678,299.834,294.269,306.253,293.895,0.005088,...,1735.93,1300.87,303.895,1.717400e-06,1.432950,0.334410,2023-04-01,2023,4,0.147807
83576,80835,387,202305,51.204,11.678,301.408,294.354,307.231,295.266,0.019854,...,1694.04,1210.34,305.717,4.468630e-07,0.317675,0.610513,2023-05-01,2023,5,0.147549
83577,81673,387,202306,51.204,11.678,302.933,296.843,309.328,292.453,0.000158,...,1555.81,1144.27,305.315,3.380950e-08,0.585810,0.947582,2023-06-01,2023,6,0.146490
83578,82511,387,202307,51.204,11.678,304.032,299.200,309.950,290.167,0.000038,...,1414.49,1043.21,303.643,4.085260e-06,1.418990,0.833237,2023-07-01,2023,7,0.148758


In [ ]:
!pip install pandas networkx


In [ ]:
import networkx as nx

# For an undirected graph
G = nx.Graph()
from geopy.distance import geodesic

In [ ]:
# Add nodes with features
for index, row in processed_dataset.iterrows():
    G.add_node(index, **row[['Lat', 'Lon', 'temperature_2m', 'temperature_2m_min',
                             'temperature_2m_max', 'dewpoint_temperature_2m', 'total_precipitation_sum',
                             'total_precipitation_min', 'total_precipitation_max', 'u_component_of_wind_10m',
                             'v_component_of_wind_10m', 'surface_net_solar_radiation_sum',
                             'total_evaporation_sum', 'soil_moisture_am', 'soil_moisture_pm', 'NDVI',
                             'EVI', 'next_month_tempreature', 'next_month_precipitation', 'year', 'month2', 'spei01']].to_dict())

print("finished creating node features")



finished creating node features


KeyboardInterrupt: ignored

In [1]:
sort_processed_dataset = processed_dataset.sort(['lat','lon',month])
sort_processed_dataset

NameError: ignored

In [ ]:
# Add edges if the points are <150km apart
for i, row_i in processed_dataset.iterrows():
    for j, row_j in processed_dataset.iterrows():
        if i < j:
            distance_km = geodesic((row_i['Lat'], row_i['Lon']), (row_j['Lat'], row_j['Lon'])).kilometers
            if distance_km < 150:
                G.add_edge(i, j)

print("finished adding edges")
